In [ ]:

%spark.pyspark

path = "/apps/hive/warehouse/eakotelnikov.db/market_events" # Здесь указываем путь до папки с паркетами (но не глубже). Проверяем столбец event_date появился. 

df = spark.read.parquet(path)

df.printSchema()

z.show(df)


#### Вывести топ категорий по количеству просмотров товаров за всё время

In [1]:
%pyspark

# вывести в виде
# +--------------------------------+----------+
# |category_code                   |view_count|
# +--------------------------------+----------+
# |null                            |20837460  |
# |electronics.smartphone          |14832387  |
# |computers.notebook              |2103024   |

from pyspark.sql.functions import *

view_filter = col("event_type") == "view"

(
    df
    .where(view_filter)
    .groupBy("category_code")
    .agg(count("event_type").alias("view_count"))
    .orderBy(col("view_count")
    .desc())
    .show(truncate=False) 
)



#### Построить гистограмму распределения цен на проданные товары за 10 октября

In [2]:
%pyspark

# результат визуализировать через z.show()

from pyspark.sql.functions import *

#from pyspark.sql.functions import year, month, dayofmonth, max, min
#from pyspark.sql.functions import col

from pyspark.sql.window import Window
from pyspark.ml.feature import Bucketizer

day_filter = col("event_date") == "2019-11-10"
event_type_filter = col("event_type") == "purchase"

(
    df
    .where(day_filter & event_type_filter)
    .select(max(df.price))
    .show()
)

bucketizer = Bucketizer(splits=[i*10 for i in range(259)], inputCol="price", outputCol="bin")

df_buck = (
            bucketizer
                    .setHandleInvalid("keep")
                    .transform(df.where(day_filter & event_type_filter))
                    .select(col("price"),col("bin"))
                    .groupBy("bin")
                    .agg(count("price")
                    .alias("price_count"))
                    .orderBy(col("bin")
                    .asc())
)                    
                    
df_buck.select(min(df_buck.bin)).show()
df_buck.select(max(df_buck.bin)).show()
z.show(df_buck)


#### Посчитать количество продаж за октябрь отдельно бренда apple и остальных

In [3]:
%pyspark

# вывести через df.show() в виде
# +--------+------+
# |is_apple| count|
# +--------+------+
# |    true| *****|
# |   false| *****|
# +--------+------+

from pyspark.sql import functions as f
from pyspark.sql.functions import *

df.printSchema()

view_filter = col("event_type") == "purchase"

(
    df
    .where(view_filter)
    .withColumn('is_apple', f.when(f.col('brand') == "apple", "true")
    .otherwise("false"))
    .groupBy("is_apple")
    .agg(count("event_type")
    .alias("count"))
    .orderBy(col("is_apple")
    .desc())
    .show()
)


#### Построить почасовой график продаж и прибыли за вторую неделю октября

In [4]:
%pyspark
# z.show(), ключ -- часы, значения -- число продаж и сумма прибыли за этот час

from pyspark.sql.functions import *

print(df.agg({"event_date": "min"}).collect()[0])
print(df.agg({"event_date": "max"}).collect()[0])

df_week = df.withColumn("week_number", date_format(to_date("event_date", "yyyy-MM-dd"), "W"))

print(df_week.agg({"week_number": "min"}).collect()[0])
print(df_week.agg({"week_number": "max"}).collect()[0])

df_week_hour = (
                df
                .withColumn("week_number", date_format(to_date("event_date", "yyyy-MM-dd"), "W"))
                .withColumn("hour", hour(col("event_time")))
                .cache()
                )

week_number_filter = col("week_number") == 2
event_type_filter = col("event_type") == "purchase"

df_filtered_by_hour_week = df_week_hour.where(week_number_filter & event_type_filter).cache()

result = (
        df_filtered_by_hour_week
        .groupBy("hour")
        .agg(count("*")
        .alias("amount_of_sales"), sum("price").alias("profit"))
        .orderBy(col("hour").asc())
        )

z.show(result)


In [5]:
%pyspark
z.show(result)

#### Построить почасовой график числа продаж и выручки, усредненный за месяц

In [6]:
%pyspark
# z.show(), ключ -- час в диапазоне от 0 до 23, значение -- усредненное за месяц число продаж в этот час на месячных данных

from pyspark.sql.functions import *
from pyspark.sql.window import Window

df.printSchema()

view_filter = col("event_type") == "purchase"
df_intermidiate = (
                    df.where(view_filter)
                    .withColumn("hour", hour(df.event_time))
                    .withColumn("month", month(df.event_time))
                    )

windowSpec = Window.partitionBy("month").orderBy("hour")
result = (
            df_intermidiate
            .withColumn("avg_price", avg(col("price")).over(windowSpec))
            .withColumn("count_prices", count(col("price")).over(windowSpec))
            .groupBy("hour")
            .agg(sum("avg_price").alias("avg_revenues_based_on_month"), count("*").alias("avg_amount_of_sales_based_on_month"))
            .orderBy("hour")
            )
            
z.show(result)
 

In [7]:
%pyspark
z.show(result)